### 「賃料」データの外れ値がRMSEの悪化に寄与している事実に対して、施策を練る

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib
from sklearn.model_selection import KFold

###訓練データの読み込み
house_age = pd.read_csv('../data/handmade_data/house_age.csv')
area_size = pd.read_csv('../data/handmade_data/area_size.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
contract_span = pd.read_csv('../data/handmade_data/contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('../data/handmade_data/contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('../data/handmade_data/reg_rent.csv')
floor_scores = pd.read_csv('../data/handmade_data/floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('../data/handmade_data/floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('../data/handmade_data/capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('../data/handmade_data/capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('../data/handmade_data/stations.csv')
minits = pd.read_csv('../data/handmade_data/minits.csv')
addresses = pd.read_csv('../data/handmade_data/addresses.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
buildings = pd.read_csv('../data/handmade_data/buildings.csv')
land_aveprice = pd.read_csv('../data/handmade_data/land_aveprice.csv')
land_aveprice.columns = ['平均地価']


rent = pd.read_csv('../data/handmade_data/rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('../data/test_data/test_house_age.csv')
test_area_size = pd.read_csv('../data/test_data/test_area_size.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_contract_span = pd.read_csv('../data/test_data/test_contract_span.csv')
test_contract_span2 = pd.read_csv('../data/test_data/test_contract_span2.csv')
test_reg_rent = pd.read_csv('../data/test_data/test_reg_rent.csv')
test_floor_scores = pd.read_csv('../data/test_data/test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('../data/test_data/test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('../data/test_data/test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('../data/test_data/test_capital_floor_scores2.csv')
test_stations = pd.read_csv('../data/test_data/test_stations.csv')
test_minits = pd.read_csv('../data/test_data/test_minits.csv')
test_addresses = pd.read_csv('../data/test_data/test_addresses.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_buildings = pd.read_csv('../data/test_data/test_buildings.csv')
test_land_aveprice = pd.read_csv('../data/test_data/test_land_aveprice.csv')
test_land_aveprice.columns = ['平均地価']

X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, land_aveprice, addresses, room_arrange[["部屋数","L","D","S"]]], axis=1)
y_train = rent
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_land_aveprice, test_addresses, test_room_arrange[["部屋数","L","D","S"]]], axis=1)
category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','S']

Duplicate key in file PosixPath('/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/matplotlib/mpl-data/matplotlibrc'), line 258 ('font.family : IPAexGothic')
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


「賃料」がわかりやすくなるように予測に用いる特徴量と合体させる

In [2]:
DATA = pd.concat([y_train, X_train],axis=1)

In [14]:
pd.set_option('display.max_rows', 150)
prime_place = DATA[DATA['賃料']>500000]
prime_place.to_csv('../data/handmade_data/over_500000yen.csv')
prime_place

,賃料,築年数,面積,契約期間,定期借家,所在階,全体の階数,最寄り駅,所要時間,平均地価,所在地,部屋数,L,D,S
274,650000,25.0,79.91,4.000000,2,30.0,38.0,172,2.0,1.840000e+06,208,3,1,1,0
666,505000,49.0,111.65,2.000000,1,53.0,53.0,23,1.0,1.408000e+06,2,3,1,1,0
671,900000,175.0,231.52,2.000000,1,3.0,1.0,267,6.0,3.751429e+06,369,5,1,1,0
1295,550000,544.0,214.78,2.000000,1,3.0,NaN,275,5.0,7.712500e+05,435,5,1,1,0
1501,1200000,222.0,163.62,5.000000,2,3.0,3.0,235,9.0,2.213333e+06,268,3,1,1,0
1712,515000,195.0,85.86,2.000000,1,13.0,34.0,13,8.0,5.746250e+06,569,2,1,1,0
1771,1450000,109.0,232.01,4.000000,2,2.0,NaN,372,5.0,7.138889e+05,493,4,1,1,1
1940,780000,23.0,112.30,2.000000,1,9.0,16.0,19,5.0,3.945000e+06,592,3,1,1,0
2047,820000,202.0,164.88,2.000000,1,2.0,7.0,152,5.0,3.793750e+06,174,3,1,1,0
2075,880000,250.0,256.68,5.000000,2,3.0,NaN,52,3.0,1.073667e+06,165,5,1,1,0


明らかな異常値を見つけたので修正を行う。レコードID:5775は、ワンルーム約20m^2で家賃月120万であり、これはあり得ない。12万の間違いとして修正を行う。

In [17]:
DATA = pd.concat([y_train, X_train],axis=1)
DATA.loc[[5775],'賃料'] = DATA.loc[[5775],'賃料']/10
DATA.loc[5775]

「賃料」は「面積」と強い相関がある。「L」「全体の階数」「D」「所在階」・・・というように相関は続く。

In [29]:
DATA.corr()['賃料']

賃料       1.000000
築年数     -0.251352
面積       0.752858
契約期間     0.191123
定期借家     0.013631
所在階      0.429444
全体の階数    0.495466
最寄り駅     0.059917
所要時間    -0.135984
平均地価     0.267642
所在地     -0.000540
部屋数      0.375435
L        0.565994
D        0.474419
S        0.196456
Name: 賃料, dtype: float64

「面積」が80m^2以下で賃料が500000以下の物件について調べた結果、やはりあまりない。

In [35]:
DATA[(DATA['賃料']>500000) & (DATA['面積']<80)]

,賃料,築年数,面積,契約期間,定期借家,所在階,全体の階数,最寄り駅,所要時間,平均地価,所在地,部屋数,L,D,S
274,650000,25.0,79.91,4.0,2,30.0,38.0,172,2.0,1.840000e+06,208,3,1,1,0
2624,520000,6.0,75.95,2.0,1,34.0,37.0,157,5.0,2.930000e+06,180,2,1,1,0
6387,680000,20.0,72.21,5.0,2,7.0,26.0,348,3.0,3.442222e+06,13,1,1,1,0
7874,510000,6.0,74.85,2.0,1,10.0,37.0,157,5.0,2.930000e+06,180,3,1,1,1
9543,590000,21.0,73.36,2.0,1,4.0,5.0,46,7.0,8.162222e+06,641,1,1,1,0
28522,630000,25.0,75.55,2.0,1,36.0,38.0,172,2.0,1.840000e+06,208,2,1,1,0
28524,535000,37.0,76.18,2.0,1,23.0,23.0,172,5.0,1.840000e+06,208,1,1,1,0


「面積」が80^2以上の物件はかなりの数ある。

In [37]:
DATA[DATA['面積']>=80]

,賃料,築年数,面積,契約期間,定期借家,所在階,全体の階数,最寄り駅,所要時間,平均地価,所在地,部屋数,L,D,S
22,346000,364.0,81.20,NaN,3,10.0,42.0,23,6.0,1.232750e+06,23,2,1,1,0
30,285000,158.0,92.17,5.0,2,4.0,11.0,31,7.0,7.090000e+05,30,2,1,1,0
33,238000,52.0,80.30,2.0,2,3.0,7.0,34,10.0,7.990000e+05,33,3,1,1,0
53,238000,365.0,93.15,2.0,1,2.0,3.0,51,12.0,6.600000e+05,51,3,1,1,0
153,255000,180.0,90.24,2.0,1,2.0,7.0,121,11.0,6.180000e+05,60,3,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31358,220000,128.0,130.83,2.0,1,2.0,NaN,149,9.0,3.510000e+05,168,4,1,1,0
31376,248000,191.0,105.27,3.0,2,2.0,1.0,131,5.0,7.496667e+05,140,2,1,1,0
31406,280000,67.0,86.14,NaN,3,7.0,9.0,228,12.0,5.116667e+05,580,4,1,1,0
31435,243000,50.0,93.25,4.0,2,2.0,NaN,258,8.0,4.855000e+05,510,3,1,1,0


「面積」が80^2以上かつリビング付きの物件の割合はほぼ１。つまり広い物件はリビングを必ず持っている。

In [47]:
len(DATA[(DATA['面積']>=80) & (DATA['L']==1)])/len(DATA[DATA['面積']>=80])

0.978391356542617

逆に80^2かつリビングなしの物件はどんなものかというと、単純に部屋の数が多い印象

In [50]:
DATA[(DATA['面積']>=80) & (DATA['L']==0)]

,賃料,築年数,面積,契約期間,定期借家,所在階,全体の階数,最寄り駅,所要時間,平均地価,所在地,部屋数,L,D,S
3189,170000,346.0,86.09,2.0,1,10.0,11.0,56,9.0,589500.000000,473,3,0,1,0
6733,148000,355.0,105.58,5.0,2,2.0,3.0,352,10.0,406500.000000,200,3,0,1,1
9019,417600,197.0,100.41,2.0,1,29.0,41.0,370,6.0,NaN,534,5,0,0,0
10432,100000,744.0,86.74,2.0,1,2.0,NaN,217,16.0,529333.333333,339,3,0,0,0
11828,230000,531.0,108.89,2.0,1,3.0,11.0,72,1.0,398000.000000,72,4,0,1,0
11994,150000,835.0,87.60,2.0,1,1.0,2.0,141,10.0,576666.666667,341,4,0,0,0
12498,130000,454.0,98.33,2.0,1,2.0,NaN,36,5.0,320500.000000,529,5,0,1,0
15259,238000,337.0,109.09,2.0,1,3.0,NaN,117,6.0,580000.000000,752,4,0,1,0
16241,150000,356.0,112.61,10.0,2,2.0,NaN,97,NaN,263000.000000,263,5,0,1,1
16667,205000,353.0,98.91,2.0,1,4.0,5.0,56,10.0,534600.000000,82,1,0,0,0


ひとまず、「面積」が80m^2以上の物件について目星をつけてみる

In [66]:
X_train['広い物件'] = 0
y_train.loc[[5775], '賃料'] = y_train.loc[[5775], '賃料']/10

for i in range(len(X_train)):
    if X_train.loc[i]['面積'] >= 80.0:
        #X_train.loc[i]['広い物件'] = 1#この代入方法だとエラーも出ずに代入に失敗する
        X_train.loc[i,'広い物件'] = 1

In [67]:
X_train[X_train['広い物件']==1]

,築年数,面積,契約期間,定期借家,所在階,全体の階数,最寄り駅,所要時間,平均地価,所在地,部屋数,L,D,S,広い物件
22,364.0,81.20,NaN,3,10.0,42.0,23,6.0,1.232750e+06,23,2,1,1,0,1
30,158.0,92.17,5.0,2,4.0,11.0,31,7.0,7.090000e+05,30,2,1,1,0,1
33,52.0,80.30,2.0,2,3.0,7.0,34,10.0,7.990000e+05,33,3,1,1,0,1
53,365.0,93.15,2.0,1,2.0,3.0,51,12.0,6.600000e+05,51,3,1,1,0,1
153,180.0,90.24,2.0,1,2.0,7.0,121,11.0,6.180000e+05,60,3,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31358,128.0,130.83,2.0,1,2.0,NaN,149,9.0,3.510000e+05,168,4,1,1,0,1
31376,191.0,105.27,3.0,2,2.0,1.0,131,5.0,7.496667e+05,140,2,1,1,0,1
31406,67.0,86.14,NaN,3,7.0,9.0,228,12.0,5.116667e+05,580,4,1,1,0,1
31435,50.0,93.25,4.0,2,2.0,NaN,258,8.0,4.855000e+05,510,3,1,1,0,1


次に、「面積」が70m^2以上に限って、「賃料」の相関を調べてみると、依然として「面積」と「賃料」の相関は強い。

In [82]:
DATA['広い物件'] = 0

for i in range(len(DATA)):
    if DATA.loc[i]['面積'] >= 70.0:
        #X_train.loc[i]['広い物件'] = 1#この代入方法だとエラーも出ずに代入に失敗する
        DATA.loc[i,'広い物件'] = 1

In [83]:
DATA_over70m2 = DATA[DATA['面積']>70.0]
DATA_over70m2

,賃料,築年数,面積,契約期間,定期借家,所在階,全体の階数,最寄り駅,所要時間,平均地価,所在地,部屋数,L,D,S,広い物件
18,310000,72.0,70.82,2.0,1,5.0,11.0,19,5.0,1.037000e+06,19,3,1,1,0,1
22,346000,364.0,81.20,NaN,3,10.0,42.0,23,6.0,1.232750e+06,23,2,1,1,0,1
30,285000,158.0,92.17,5.0,2,4.0,11.0,31,7.0,7.090000e+05,30,2,1,1,0,1
33,238000,52.0,80.30,2.0,2,3.0,7.0,34,10.0,7.990000e+05,33,3,1,1,0,1
36,146000,260.0,70.04,2.0,1,2.0,3.0,7,10.0,3.485000e+05,36,3,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31376,248000,191.0,105.27,3.0,2,2.0,1.0,131,5.0,7.496667e+05,140,2,1,1,0,1
31406,280000,67.0,86.14,NaN,3,7.0,9.0,228,12.0,5.116667e+05,580,4,1,1,0,1
31429,308000,139.0,71.29,2.0,1,21.0,58.0,2,5.0,1.580000e+06,169,1,1,1,0,1
31435,243000,50.0,93.25,4.0,2,2.0,NaN,258,8.0,4.855000e+05,510,3,1,1,0,1


In [84]:
DATA_over70m2.corr()['賃料']

賃料       1.000000
築年数     -0.204320
面積       0.545984
契約期間     0.116142
定期借家     0.040640
所在階      0.265245
全体の階数    0.195074
最寄り駅     0.141049
所要時間    -0.294123
平均地価     0.466068
所在地      0.025937
部屋数     -0.097555
L        0.113633
D        0.041397
S        0.084708
広い物件          NaN
Name: 賃料, dtype: float64

「面積」が100m^2以上で、「賃料」が500000以上のもの

In [87]:
print(len(DATA[(DATA['賃料']>500000) & (DATA['面積']>100.0)]))
DATA[(DATA['賃料']>500000) & (DATA['面積']>100.0)]

88


,賃料,築年数,面積,契約期間,定期借家,所在階,全体の階数,最寄り駅,所要時間,平均地価,所在地,部屋数,L,D,S,広い物件
666,505000,49.0,111.65,2.000000,1,53.0,53.0,23,1.0,1.408000e+06,2,3,1,1,0,1
671,900000,175.0,231.52,2.000000,1,3.0,1.0,267,6.0,3.751429e+06,369,5,1,1,0,1
1295,550000,544.0,214.78,2.000000,1,3.0,NaN,275,5.0,7.712500e+05,435,5,1,1,0,1
1501,1200000,222.0,163.62,5.000000,2,3.0,3.0,235,9.0,2.213333e+06,268,3,1,1,0,1
1771,1450000,109.0,232.01,4.000000,2,2.0,NaN,372,5.0,7.138889e+05,493,4,1,1,1,1
1940,780000,23.0,112.30,2.000000,1,9.0,16.0,19,5.0,3.945000e+06,592,3,1,1,0,1
2047,820000,202.0,164.88,2.000000,1,2.0,7.0,152,5.0,3.793750e+06,174,3,1,1,0,1
2075,880000,250.0,256.68,5.000000,2,3.0,NaN,52,3.0,1.073667e+06,165,5,1,1,0,1
2266,700000,413.0,152.07,2.000000,2,5.0,7.0,13,9.0,5.746250e+06,569,2,1,1,1,1
2909,700000,20.0,103.04,3.000000,2,3.0,9.0,235,8.0,1.790000e+06,451,2,1,1,0,1


### 「面積」が100m^2以上という特徴量を作成し、モデルを動かしてみる。

In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import scipy as sp
import lightgbm as lgb
import category_encoders as ce
# import mojimoji
import re
from cmath import nan
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import optuna
from sklearn.metrics import mean_squared_error
import japanize_matplotlib
from sklearn.model_selection import KFold

###訓練データの読み込み
house_age = pd.read_csv('../data/handmade_data/house_age.csv')
area_size = pd.read_csv('../data/handmade_data/area_size.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
contract_span = pd.read_csv('../data/handmade_data/contract_span.csv')#欠損値をゼロ埋めした契約期間
contract_span2 = pd.read_csv('../data/handmade_data/contract_span2.csv')#欠損値を欠損値のままにした契約期間
reg_rent = pd.read_csv('../data/handmade_data/reg_rent.csv')
floor_scores = pd.read_csv('../data/handmade_data/floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「所在階」
floor_scores2 = pd.read_csv('../data/handmade_data/floor_scores2.csv')#「所在階」と「全体の階数」が独立している「所在階」
Floor_scores = pd.read_csv('../data/handmade_data/capital_floor_scores.csv')#「所在階」と「全体の階数」が連動して欠損値になる場合のある「全体の階数」
Floor_scores2 = pd.read_csv('../data/handmade_data/capital_floor_scores2.csv')#「所在階」と「全体の階数」が独立している「全体の階数」
stations = pd.read_csv('../data/handmade_data/stations.csv')
minits = pd.read_csv('../data/handmade_data/minits.csv')
addresses = pd.read_csv('../data/handmade_data/addresses.csv')
room_arrange = pd.read_csv('../data/handmade_data/room_arrange.csv')
buildings = pd.read_csv('../data/handmade_data/buildings.csv')
land_aveprice = pd.read_csv('../data/handmade_data/land_aveprice.csv')
land_aveprice.columns = ['平均地価']


rent = pd.read_csv('../data/handmade_data/rent.csv')


##テストデータの読み込み
test_house_age = pd.read_csv('../data/test_data/test_house_age.csv')
test_area_size = pd.read_csv('../data/test_data/test_area_size.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_contract_span = pd.read_csv('../data/test_data/test_contract_span.csv')
test_contract_span2 = pd.read_csv('../data/test_data/test_contract_span2.csv')
test_contract_span2.columns = ['契約期間']
test_reg_rent = pd.read_csv('../data/test_data/test_reg_rent.csv')
test_floor_scores = pd.read_csv('../data/test_data/test_floor_scores.csv')
test_floor_scores2 = pd.read_csv('../data/test_data/test_floor_scores2.csv')
test_Floor_scores = pd.read_csv('../data/test_data/test_capital_floor_scores.csv')
test_Floor_scores2 = pd.read_csv('../data/test_data/test_capital_floor_scores2.csv')
test_stations = pd.read_csv('../data/test_data/test_stations.csv')
test_minits = pd.read_csv('../data/test_data/test_minits.csv')
test_addresses = pd.read_csv('../data/test_data/test_addresses.csv')
test_room_arrange = pd.read_csv('../data/test_data/test_room_arrange.csv')
test_buildings = pd.read_csv('../data/test_data/test_buildings.csv')
test_land_aveprice = pd.read_csv('../data/test_data/test_land_aveprice.csv')
test_land_aveprice.columns = ['平均地価']

X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, land_aveprice, addresses, room_arrange[["部屋数","L","D","S"]]], axis=1)
y_train = rent
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_land_aveprice, test_addresses, test_room_arrange[["部屋数","L","D","S"]]], axis=1)
category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','S']

In [129]:
X_train['100m^2以上'] = 0
X_test['100m^2以上'] = 0
y_train.loc[[5775], '賃料'] = y_train.loc[[5775], '賃料']/10

for i in range(len(X_train)):
    if X_train.loc[i]['面積'] >= 100.0:
        #X_train.loc[i]['広い物件'] = 1#この代入方法だとエラーも出ずに代入に失敗する
        X_train.loc[i,'100m^2以上'] = 1

for i in range(len(X_test)):
    if X_test.loc[i]['面積'] >= 100.0:
        X_test.loc[i, '100m^2以上'] = 1

over100m2 = X_train['100m^2以上']
over100m2.to_csv('../data/handmade_data/over100m2.csv')
test_over100m2 = X_test['100m^2以上']
test_over100m2.to_csv('../data/test_data/test_over100m2.csv')

In [131]:
X_test[X_test['面積']>90.0]

,築年数,面積,契約期間,定期借家,所在階,全体の階数,最寄り駅,所要時間,平均地価,所在地,部屋数,L,D,S,100m^2以上
17,415.0,112.96,4.0,3,7.0,14.0,89,5.0,1.436667e+06,78,3,1,1,0,1
38,139.0,91.38,NaN,1,23.0,58.0,2,5.0,1.580000e+06,169,2,1,1,0,0
109,65.0,105.26,2.0,2,2.0,NaN,154,17.0,5.396667e+05,185,3,1,1,0,1
132,376.0,102.70,2.0,2,4.0,4.0,77,10.0,5.270000e+05,79,3,1,1,0,1
199,197.0,93.22,2.0,2,37.0,41.0,124,4.0,NaN,534,2,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30828,476.0,120.00,2.0,2,2.0,NaN,292,7.0,4.091429e+05,76,6,1,1,0,1
30923,184.0,104.96,2.0,3,10.0,11.0,290,6.0,1.645500e+06,26,1,1,1,0,1
31119,301.0,93.97,2.0,2,29.0,33.0,120,9.0,NaN,506,2,1,1,0,0
31158,197.0,118.38,2.0,2,5.0,6.0,8,12.0,7.816667e+05,254,3,1,1,1,1


GBDTを動かしてみる

In [132]:
category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','S', '100m^2以上']

n_split = 5
cv = KFold(n_splits=n_split, shuffle=True, random_state=0)

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    params = {
        'objective':'regression',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda':  2.903886846508796e-05,
        'max_bin': 288,
        'num_leaves': 116
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=1000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)
# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
df_importance = pd.DataFrame(importance)
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001102 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2162
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 15
[LightGBM] [Info] Start training from score 118516.069987
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 48636.7	valid_1's rmse: 50260.4
[20]	training's rmse: 34353.3	valid_1's rmse: 38117.8
[30]	training's rmse: 25685	valid_1's rmse: 31116.3
[40]	training's rmse: 20777.4	valid_1's rmse: 27333.6
[50]	training's rmse: 17492.5	valid_1's rmse: 24937
[60]	training's rmse: 15587.3	valid_1's rmse: 23592.1
[70]	training's rmse: 14245.5	valid_1's rmse: 22738.5
[80]	training's rmse: 13253.4	valid_1's rmse: 22109.3
[90]	training's rmse: 12465	valid_1's rmse: 21659.5
[100]	training's rmse: 11831.9	valid_1's rmse

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

[20]	training's rmse: 34648.6	valid_1's rmse: 35830.6
[30]	training's rmse: 26543.7	valid_1's rmse: 28412.4
[40]	training's rmse: 21850.7	valid_1's rmse: 24414.9
[50]	training's rmse: 18820.8	valid_1's rmse: 21796.6
[60]	training's rmse: 17046.1	valid_1's rmse: 20286
[70]	training's rmse: 15755.4	valid_1's rmse: 19394.9
[80]	training's rmse: 14768.4	valid_1's rmse: 18770.8
[90]	training's rmse: 14000.9	valid_1's rmse: 18436.6
[100]	training's rmse: 13320.2	valid_1's rmse: 18105.8
[110]	training's rmse: 12743.5	valid_1's rmse: 17910.4
[120]	training's rmse: 12268.6	valid_1's rmse: 17755.3
[130]	training's rmse: 11781.2	valid_1's rmse: 17632.6
[140]	training's rmse: 11376.9	valid_1's rmse: 17525.9
[150]	training's rmse: 11004.6	valid_1's rmse: 17383.4
[160]	training's rmse: 10672.6	valid_1's rmse: 17319.9
[170]	training's rmse: 10356.6	valid_1's rmse: 17284.5
[180]	training's rmse: 10080.9	valid_1's rmse: 17223.4
[190]	training's rmse: 9806.02	valid_1's rmse: 17176.7
[200]	training's rms

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

...

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

[20]	training's rmse: 34691.6	valid_1's rmse: 35403.1
[30]	training's rmse: 26285.3	valid_1's rmse: 28519.4
[40]	training's rmse: 21569.9	valid_1's rmse: 24885.9
[50]	training's rmse: 18365.4	valid_1's rmse: 23033.9
[60]	training's rmse: 16484.4	valid_1's rmse: 22155.9
[70]	training's rmse: 15292.5	valid_1's rmse: 21654.4
[80]	training's rmse: 14403.6	valid_1's rmse: 21177.9
[90]	training's rmse: 13705	valid_1's rmse: 20731.3
[100]	training's rmse: 13134.7	valid_1's rmse: 20402.8
[110]	training's rmse: 12645.3	valid_1's rmse: 20060.5
[120]	training's rmse: 12209.6	valid_1's rmse: 19852.9
[130]	training's rmse: 11815.4	valid_1's rmse: 19681.8
[140]	training's rmse: 11466.9	valid_1's rmse: 19615.7
[150]	training's rmse: 11140.7	valid_1's rmse: 19530.3
[160]	training's rmse: 10836.2	valid_1's rmse: 19451.6
[170]	training's rmse: 10560	valid_1's rmse: 19424.5
[180]	training's rmse: 10288.6	valid_1's rmse: 19337
[190]	training's rmse: 10041.8	valid_1's rmse: 19315.8
[200]	training's rmse: 9

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

[20]	training's rmse: 34811.1	valid_1's rmse: 37028.9
[30]	training's rmse: 26671.8	valid_1's rmse: 28598.4
[40]	training's rmse: 22117.3	valid_1's rmse: 24039.4
[50]	training's rmse: 19064.5	valid_1's rmse: 21442.4
[60]	training's rmse: 17221	valid_1's rmse: 20018.4
[70]	training's rmse: 16008	valid_1's rmse: 19188.4
[80]	training's rmse: 15044.6	valid_1's rmse: 18682.4
[90]	training's rmse: 14311.3	valid_1's rmse: 18345.9
[100]	training's rmse: 13730	valid_1's rmse: 17986.7
[110]	training's rmse: 13216.8	valid_1's rmse: 17744.9
[120]	training's rmse: 12786.2	valid_1's rmse: 17580.5
[130]	training's rmse: 12397.7	valid_1's rmse: 17364.3
[140]	training's rmse: 12035.9	valid_1's rmse: 17262.9
[150]	training's rmse: 11727.6	valid_1's rmse: 17138.9
[160]	training's rmse: 11460.5	valid_1's rmse: 17007.6
[170]	training's rmse: 11201.3	valid_1's rmse: 16906
[180]	training's rmse: 10970.9	valid_1's rmse: 16794.8
[190]	training's rmse: 10764.4	valid_1's rmse: 16700.7
[200]	training's rmse: 105

,importance
面積,4.452362e+14
L,4.120082e+14
平均地価,2.082377e+14
最寄り駅,9.001409e+13
築年数,7.439839e+13
所在地,3.986304e+13
全体の階数,3.422603e+13
契約期間,1.907748e+13
所在階,9.622100e+12
所要時間,6.611033e+12


チューニングを行ってみる

In [137]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, land_aveprice, addresses, room_arrange[["部屋数","L","D",'S']], over100m2], axis=1)
y_train = rent
y_train.loc[[5775], '賃料'] = y_train.loc[[5775], '賃料']/10
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_land_aveprice, test_addresses, test_room_arrange[["部屋数","L","D","S"]], test_over100m2], axis=1)
category_lists = ['定期借家', '最寄り駅', '所在地', 'L', 'D', 'S', '100m^2以上']

cv = KFold(n_splits=5, shuffle=True, random_state=0)
rmse_scores = []
X_train['KFold'] = -1
#foldはvalidの対象となる値で０から４
for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_train.loc[valid_index,'KFold'] = fold_id
    y_train.loc[valid_index,'KFold'] = fold_id

def objective(trial):
    #foldはvalidの対象となる値で０から４
    for fold in range(5):
        X_tr = X_train[X_train['KFold']!=fold].drop('KFold',axis=1)
        X_val = X_train[X_train['KFold']==fold].drop('KFold',axis=1)
        y_tr = y_train[y_train['KFold']!=fold].drop('KFold',axis=1)
        y_val = y_train[y_train['KFold']==fold].drop('KFold',axis=1)

        lgb_train = lgb.Dataset(X_tr, y_tr)
        lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)
        params = {
            'objective':'regression',
            'metrics':'rmse',
            'learning_rate':0.05,
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0000001, 0.0001, log=True),
            'max_bin': trial.suggest_int('max_bin', 255, 600),
            'num_leaves': trial.suggest_int('num_leaves', 50, 128),
        }

        model = lgb.train(
                            params,
                            lgb_train, 
                            valid_sets=[lgb_train, lgb_eval], 
                            verbose_eval=10, 
                            num_boost_round=1000, 
                            early_stopping_rounds=10,
                            categorical_feature = category_lists
                            )

        y_pred_val = model.predict(X_val, num_iteration=model.best_iteration)
        rmse_score = mean_squared_error(y_val, y_pred_val, squared=False)
    loss = np.mean(rmse_score)
    return loss

In [138]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

[I 2022-09-15 11:37:14,654] A new study created in memory with name: no-name-ede1a129-7819-4bda-90f4-960bb7aee031
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future 

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001467 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2824
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 15
[LightGBM] [Info] Start training from score 118516.069987
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 48789.7	valid_1's rmse: 50223.2
[20]	training's rmse: 34455.9	valid_1's rmse: 37916.3
[30]	training's rmse: 26131.4	valid_1's rmse: 31215.8
[40]	training's rmse: 20982.9	valid_1's rmse: 27424.8
[50]	training's rmse: 17746.4	valid_1's rmse: 25091.9
[60]	training's rmse: 15751.7	valid_1's rmse: 23720.7
[70]	training's rmse: 14387.1	valid_1's rmse: 22944.3
[80]	training's rmse: 13389.1	valid_1's rmse: 22358.9
[90]	training's rmse: 12619.2	valid_1's rmse: 21933
[100]	training's rmse: 11986.4	valid_1's 

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001192 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2823
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 15
[LightGBM] [Info] Start training from score 118431.054159
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 48987.7	valid_1's rmse: 49549
[20]	training's rmse: 34969.4	valid_1's rmse: 35966.9
[30]	training's rmse: 27033.6	valid_1's rmse: 28626.7
[40]	training's rmse: 22331.9	valid_1's rmse: 24334.6
[50]	training's rmse: 19336	valid_1's rmse: 21539.5
[60]	training's rmse: 17574.9	valid_1's rmse: 20193.3
[70]	training's rmse: 16408.2	valid_1's rmse: 19332.1
[80]	training's rmse: 15560.3	valid_1's rmse: 18808.7
[90]	training's rmse: 14858.5	valid_1's rmse: 18599.4
[100]	training's rmse: 14277.5	valid_1's rm

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

...

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002416 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2983
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 15
[LightGBM] [Info] Start training from score 118276.117989
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 48936.8	valid_1's rmse: 47993.7
[20]	training's rmse: 34600.9	valid_1's rmse: 35497
[30]	training's rmse: 26228.7	valid_1's rmse: 28387.9
[40]	training's rmse: 21400.5	valid_1's rmse: 24737
[50]	training's rmse: 18297.2	valid_1's rmse: 22863
[60]	training's rmse: 16565.9	valid_1's rmse: 21788.8
[70]	training's rmse: 15448.9	valid_1's rmse: 21133.9
[80]	training's rmse: 14684.6	valid_1's rmse: 20703.7
[90]	training's rmse: 14090.3	valid_1's rmse: 20352.1
[100]	training's rmse: 13604.5	valid_1's rmse

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

[10]	training's rmse: 48454.5	valid_1's rmse: 50877.2
[20]	training's rmse: 34627.4	valid_1's rmse: 36918.5
[30]	training's rmse: 26622	valid_1's rmse: 28660.4
[40]	training's rmse: 21882.1	valid_1's rmse: 23891.8
[50]	training's rmse: 18711.1	valid_1's rmse: 20918.6
[60]	training's rmse: 16929.3	valid_1's rmse: 19569
[70]	training's rmse: 15752.1	valid_1's rmse: 18768.5
[80]	training's rmse: 14876	valid_1's rmse: 18389.3
[90]	training's rmse: 14171.6	valid_1's rmse: 18159.3
[100]	training's rmse: 13691.6	valid_1's rmse: 17975.3
[110]	training's rmse: 13280.7	valid_1's rmse: 17860.2
[120]	training's rmse: 12897.8	valid_1's rmse: 17797.7
[130]	training's rmse: 12566.2	valid_1's rmse: 17728.7
[140]	training's rmse: 12289.8	valid_1's rmse: 17676.9
[150]	training's rmse: 12014	valid_1's rmse: 17616
[160]	training's rmse: 11760.8	valid_1's rmse: 17559
[170]	training's rmse: 11559.8	valid_1's rmse: 17533.4
[180]	training's rmse: 11342.2	valid_1's rmse: 17504.5
[190]	training's rmse: 11147.3	

[I 2022-09-15 11:45:06,746] Trial 39 finished with value: 17221.468097502388 and parameters: {'reg_lambda': 5.543941231681784e-06, 'max_bin': 560, 'num_leaves': 104}. Best is trial 22 with value: 15882.106756402674.


In [139]:
study.best_params

{'reg_lambda': 2.903886846508796e-05, 'max_bin': 288, 'num_leaves': 116}

チューニング結果を用いてモデルを動かしてみる

In [145]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, land_aveprice, addresses, room_arrange[["部屋数","L","D",'S']], over100m2], axis=1)
y_train = pd.read_csv('../data/handmade_data/rent.csv')
y_train.loc[[5775], '賃料'] = y_train.loc[[5775], '賃料']/10
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_land_aveprice, test_addresses, test_room_arrange[["部屋数","L","D","S"]], test_over100m2], axis=1)
category_lists = ['定期借家', '最寄り駅', '所在地', 'L', 'D', 'S', '100m^2以上']

n_split = 5
cv = KFold(n_splits=n_split, shuffle=True, random_state=0)

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    params = {
        'objective':'regression',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda':  2.903886846508796e-05,
        'max_bin': 288,
        'num_leaves': 116
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=1000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)
# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
df_importance = pd.DataFrame(importance)
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001508 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2162
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 15
[LightGBM] [Info] Start training from score 118516.069987
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 48636.7	valid_1's rmse: 50260.4
[20]	training's rmse: 34353.3	valid_1's rmse: 38117.8
[30]	training's rmse: 25685	valid_1's rmse: 31116.3
[40]	training's rmse: 20777.4	valid_1's rmse: 27333.6
[50]	training's rmse: 17492.5	valid_1's rmse: 24937
[60]	training's rmse: 15587.3	valid_1's rmse: 23592.1
[70]	training's rmse: 14245.5	valid_1's rmse: 22738.5
[80]	training's rmse: 13253.4	valid_1's rmse: 22109.3
[90]	training's rmse: 12465	valid_1's rmse: 21659.5
[100]	training's rmse: 11831.9	valid_1's rmse

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

[10]	training's rmse: 48699.1	valid_1's rmse: 49226.4
[20]	training's rmse: 34648.6	valid_1's rmse: 35830.6
[30]	training's rmse: 26543.7	valid_1's rmse: 28412.4
[40]	training's rmse: 21850.7	valid_1's rmse: 24414.9
[50]	training's rmse: 18820.8	valid_1's rmse: 21796.6
[60]	training's rmse: 17046.1	valid_1's rmse: 20286
[70]	training's rmse: 15755.4	valid_1's rmse: 19394.9
[80]	training's rmse: 14768.4	valid_1's rmse: 18770.8
[90]	training's rmse: 14000.9	valid_1's rmse: 18436.6
[100]	training's rmse: 13320.2	valid_1's rmse: 18105.8
[110]	training's rmse: 12743.5	valid_1's rmse: 17910.4
[120]	training's rmse: 12268.6	valid_1's rmse: 17755.3
[130]	training's rmse: 11781.2	valid_1's rmse: 17632.6
[140]	training's rmse: 11376.9	valid_1's rmse: 17525.9
[150]	training's rmse: 11004.6	valid_1's rmse: 17383.4
[160]	training's rmse: 10672.6	valid_1's rmse: 17319.9
[170]	training's rmse: 10356.6	valid_1's rmse: 17284.5
[180]	training's rmse: 10080.9	valid_1's rmse: 17223.4
[190]	training's rmse

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

...

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001318 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2159
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 15
[LightGBM] [Info] Start training from score 118280.850532
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 49083.7	valid_1's rmse: 48054.1
[20]	training's rmse: 34691.6	valid_1's rmse: 35403.1
[30]	training's rmse: 26285.3	valid_1's rmse: 28519.4
[40]	training's rmse: 21569.9	valid_1's rmse: 24885.9
[50]	training's rmse: 18365.4	valid_1's rmse: 23033.9
[60]	training's rmse: 16484.4	valid_1's rmse: 22155.9
[70]	training's rmse: 15292.5	valid_1's rmse: 21654.4
[80]	training's rmse: 14403.6	valid_1's rmse: 21177.9
[90]	training's rmse: 13705	valid_1's rmse: 20731.3
[100]	training's rmse: 13134.7	valid_1's 

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_e

[10]	training's rmse: 48495.2	valid_1's rmse: 51085.3
[20]	training's rmse: 34811.1	valid_1's rmse: 37028.9
[30]	training's rmse: 26671.8	valid_1's rmse: 28598.4
[40]	training's rmse: 22117.3	valid_1's rmse: 24039.4
[50]	training's rmse: 19064.5	valid_1's rmse: 21442.4
[60]	training's rmse: 17221	valid_1's rmse: 20018.4
[70]	training's rmse: 16008	valid_1's rmse: 19188.4
[80]	training's rmse: 15044.6	valid_1's rmse: 18682.4
[90]	training's rmse: 14311.3	valid_1's rmse: 18345.9
[100]	training's rmse: 13730	valid_1's rmse: 17986.7
[110]	training's rmse: 13216.8	valid_1's rmse: 17744.9
[120]	training's rmse: 12786.2	valid_1's rmse: 17580.5
[130]	training's rmse: 12397.7	valid_1's rmse: 17364.3
[140]	training's rmse: 12035.9	valid_1's rmse: 17262.9
[150]	training's rmse: 11727.6	valid_1's rmse: 17138.9
[160]	training's rmse: 11460.5	valid_1's rmse: 17007.6
[170]	training's rmse: 11201.3	valid_1's rmse: 16906
[180]	training's rmse: 10970.9	valid_1's rmse: 16794.8
[190]	training's rmse: 1076

,importance
面積,4.452362e+14
L,4.120082e+14
平均地価,2.082377e+14
最寄り駅,9.001409e+13
築年数,7.439839e+13
所在地,3.986304e+13
全体の階数,3.422603e+13
契約期間,1.907748e+13
所在階,9.622100e+12
所要時間,6.611033e+12


「面積」が75m^2以上100未満にもフラグを立ててみる

In [149]:
X_train['75m^2以上100m^2未満'] = 0
X_test['75m^2以上100m^2未満'] = 0
y_train = pd.read_csv('../data/handmade_data/rent.csv')
y_train.loc[[5775], '賃料'] = y_train.loc[[5775], '賃料']/10

for i in range(len(X_train)):
    if X_train.loc[i]['面積'] >= 70.0 and X_train.loc[i]['面積'] < 100.0:
        #X_train.loc[i]['広い物件'] = 1#この代入方法だとエラーも出ずに代入に失敗する
        X_train.loc[i,'75m^2以上100m^2未満'] = 1

for i in range(len(X_test)):
    if X_test.loc[i]['面積'] >= 70.0 and X_test.loc[i]['面積'] < 100.0:
        X_test.loc[i, '75m^2以上100m^2未満'] = 1

bt75and100m2 = X_train['75m^2以上100m^2未満']
bt75and100m2.to_csv('../data/handmade_data/bt75and100m2.csv')
test_bt75and100m2 = X_test['75m^2以上100m^2未満']
test_bt75and100m2.to_csv('../data/test_data/test_bt75and100m2.csv')

In [154]:
X_train = pd.concat([house_age, area_size, contract_span2, reg_rent, floor_scores2, Floor_scores2, stations, minits, land_aveprice, addresses, room_arrange[["部屋数","L","D",'S']], over100m2, bt75and100m2], axis=1)
y_train = pd.read_csv('../data/handmade_data/rent.csv')
y_train.loc[[5775], '賃料'] = y_train.loc[[5775], '賃料']/10
X_test = pd.concat([test_house_age, test_area_size, test_contract_span2, test_reg_rent, test_floor_scores2, test_Floor_scores2, test_stations, test_minits, test_land_aveprice, test_addresses, test_room_arrange[["部屋数","L","D","S"]], test_over100m2, test_bt75and100m2], axis=1)

category_lists = ['定期借家','最寄り駅', '所在地', 'L', 'D','S', '100m^2以上', '75m^2以上100m^2未満']

n_split = 5
cv = KFold(n_splits=n_split, shuffle=True, random_state=0)

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index,:]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train.loc[train_index]
    y_val = y_train.loc[valid_index]

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    params = {
        'objective':'regression',
        'metrics':'rmse',
        'learning_rate':0.05,
        'reg_lambda':  2.903886846508796e-05,
        'max_bin': 288,
        'num_leaves': 116
    }

    model = lgb.train(
                        params,
                        lgb_train, 
                        valid_sets=[lgb_train, lgb_eval], 
                        verbose_eval=10, 
                        num_boost_round=1000, 
                        early_stopping_rounds=10,
                        categorical_feature = category_lists
                        )

y_pred = model.predict(X_test, num_iteration=model.best_iteration)
# feature importanceを表示
importance = pd.DataFrame(model.feature_importance(importance_type = "gain"), index=X_train.columns, columns=['importance'])
df_importance = pd.DataFrame(importance)
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', '75m^2以上100m^2未満', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001679 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2165
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 16
[LightGBM] [Info] Start training from score 118516.069987
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 48636.7	valid_1's rmse: 50260.4
[20]	training's rmse: 34353.4	valid_1's rmse: 38117.6
[30]	training's rmse: 25744.1	valid_1's rmse: 31208.7
[40]	training's rmse: 20773.7	valid_1's rmse: 27422.9
[50]	training's rmse: 17493.4	valid_1's rmse: 25044.9
[60]	training's rmse: 15596.9	valid_1's rmse: 23695.6
[70]	training's rmse: 14239.9	valid_1's rmse: 22827.4
[80]	training's rmse: 13277	valid_1's rmse: 22177.2
[90]	training's rmse: 12485	valid_1's rmse: 21656.7
[100]	training's rmse: 11844	valid_1's rmse

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', '75m^2以上100m^2未満', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000818 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2164
[LightGBM] [Info] Number of data points in the train set: 25176, number of used features: 16
[LightGBM] [Info] Start training from score 118435.786702
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 48699.1	valid_1's rmse: 49226.4
[20]	training's rmse: 34670.6	valid_1's rmse: 35828.7
[30]	training's rmse: 26496.5	valid_1's rmse: 28314.2
[40]	training's rmse: 21900.3	valid_1's rmse: 24424.1
[50]	training's rmse: 18818.4	valid_1's rmse: 21831.3
[60]	training's rmse: 17005.9	valid_1's rmse: 20212.8
[70]	training's rmse: 15739	valid_1's rmse: 19270.3
[80]	training's rmse: 14719.2	valid_1's rmse: 18678.7
[90]	training's rmse: 13936.7	valid_1's rmse: 18294.9
[100]	training's rmse: 13300.1	valid_1's 

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', '75m^2以上100m^2未満', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_

...

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', '75m^2以上100m^2未満', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_

[10]	training's rmse: 49083.7	valid_1's rmse: 48054.1
[20]	training's rmse: 34691.6	valid_1's rmse: 35403.1
[30]	training's rmse: 26285.3	valid_1's rmse: 28519.5
[40]	training's rmse: 21471.4	valid_1's rmse: 24834.4
[50]	training's rmse: 18318.4	valid_1's rmse: 22929
[60]	training's rmse: 16435.3	valid_1's rmse: 22093.2
[70]	training's rmse: 15239	valid_1's rmse: 21596.5
[80]	training's rmse: 14335.8	valid_1's rmse: 21112.5
[90]	training's rmse: 13637.1	valid_1's rmse: 20628.3
[100]	training's rmse: 13077.2	valid_1's rmse: 20298.3
[110]	training's rmse: 12597	valid_1's rmse: 19962.8
[120]	training's rmse: 12147.1	valid_1's rmse: 19741.7
[130]	training's rmse: 11764.3	valid_1's rmse: 19593.4
[140]	training's rmse: 11421	valid_1's rmse: 19506.9
[150]	training's rmse: 11087.6	valid_1's rmse: 19487.8
[160]	training's rmse: 10783.9	valid_1's rmse: 19366.7
[170]	training's rmse: 10502.8	valid_1's rmse: 19310.8
[180]	training's rmse: 10247.3	valid_1's rmse: 19216.6
[190]	training's rmse: 1001

/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['100m^2以上', '75m^2以上100m^2未満', 'D', 'L', 'S', '定期借家', '所在地', '最寄り駅']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/Users/koshidatatsuo/python/signate/mynabi/venv/lib/python3.10/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_

[20]	training's rmse: 34817	valid_1's rmse: 37004.4
[30]	training's rmse: 26635.1	valid_1's rmse: 28565.5
[40]	training's rmse: 22110.3	valid_1's rmse: 23929.1
[50]	training's rmse: 19060.9	valid_1's rmse: 21317.9
[60]	training's rmse: 17240	valid_1's rmse: 19918.6
[70]	training's rmse: 16023.4	valid_1's rmse: 19111.8
[80]	training's rmse: 15068.5	valid_1's rmse: 18550.7
[90]	training's rmse: 14330.4	valid_1's rmse: 18188.6
[100]	training's rmse: 13750.3	valid_1's rmse: 17873.7
[110]	training's rmse: 13258.3	valid_1's rmse: 17609.8
[120]	training's rmse: 12831.4	valid_1's rmse: 17366.6
[130]	training's rmse: 12430.6	valid_1's rmse: 17246.2
[140]	training's rmse: 12079	valid_1's rmse: 17085
[150]	training's rmse: 11767	valid_1's rmse: 16953.6
[160]	training's rmse: 11486.7	valid_1's rmse: 16853
[170]	training's rmse: 11232.4	valid_1's rmse: 16740.4
[180]	training's rmse: 10981.8	valid_1's rmse: 16642.4
[190]	training's rmse: 10750.5	valid_1's rmse: 16557.9
[200]	training's rmse: 10552.4

,importance
面積,4.475731e+14
L,4.120296e+14
平均地価,2.092252e+14
最寄り駅,9.157513e+13
築年数,7.601341e+13
所在地,3.831284e+13
全体の階数,3.392870e+13
契約期間,1.861676e+13
所在階,9.916454e+12
所要時間,7.036154e+12
